In [ ]:
import pandas as pd
from wordcloud import WordCloud
from nltk.corpus import stopwords
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import RegexpTokenizer


# 1. Wordclouds
## Loading the data

In [ ]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
emails.head()

First we inspect what data we are given. For the wordcloud it would be wise to use only the "ExtractedBodyText" column.

In [ ]:
emails.columns.values

## Removing emails without body text

In [ ]:
emails.size #Size of the dataset before

In [ ]:
emails = emails[pd.notnull(emails['ExtractedBodyText'])]
emails.size #Size of the dataset after

## Generating the wordcloud

Converting the ExtractedBodyText into strings we can use for the wordcloud. Note that we tell WordCloud to not remove any WordCloud for this first graph.

In [ ]:
text = emails.ExtractedBodyText.to_string();

In [ ]:
wordcloud = WordCloud(stopwords=['']).generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## Improving the wordcloud

To improve the wordcloud we will go through the process of removing stopwords. Then we will make sure to have only alphabetical characters and use stemming to convert words into their original base form.

In [ ]:
stemmer = PorterStemmer()
sw = stopwords.words("english")

In [ ]:
def tokenize(text):
    tokens = nltk.word_tokenize(text) # Tokenizing the text
    tokens = [i for i in tokens if i.isalpha()] #Keeping only strings with alphabetic characters
    tokens = [i for i in tokens if i.lower() not in sw] #Removing stopwords
    stems = stem_tokens(tokens, stemmer) #Stemming the words
    return stems  #returns a list

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed 

In [ ]:
improvedText = str(tokenize(text)).replace("'", "") #Convert from list to string object without single quotes

In [ ]:
wc = WordCloud().generate(improvedText)
plt.imshow(wc)
plt.axis("off")
plt.show()

When comparing the two wordclouds, we see that they are largely similar if we don't take stopwords into account. In the first cloud we have 'nFriday' and 'nSunday' as backslashes are converted to 'n'.  In the second cloud some words are shortened to their stems such as 'Novemb' and 'relea'.

# 2. Sentiment analysis

# 3. Topic modeling

Using the models.ldamodel module from the gensim library, run topic modeling over the corpus. Explore different numbers of topics (varying from 5 to 50), and settle for the parameter which returns topics that you consider to be meaningful at first sight.

Start by important gensim libraries.

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora
import pickle

Read the data, keep only the ExtractedBodyText cells and remove null values.

In [ ]:
emails = pd.DataFrame(pd.read_csv('hillary-clinton-emails/Emails.csv')['ExtractedBodyText'].dropna())

Next, use the same functions defined in the first exercice to remove the stopwords, remove numbers and stem tokens. Note that we also lower all tokens.

In [ ]:
emails['ExtractedBodyText'] = emails.ExtractedBodyText.apply(lambda x: [word.lower() for word in tokenize(x) if word not in sw])

Transform the tokenized emails into a list of lists of tokens.

In [ ]:
texts = emails.ExtractedBodyText.tolist()
dictionary = corpora.Dictionary(texts)

Transform the list of lists of tokens into a corpus (a list of Bag of Words), and then train the Latent Dirichlet Allocation model on the corpus.

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

Now, make a function to print the topics in a concise way, for a better readability.

In [ ]:
def format_topics(topics):
    for topic in topics:
        print("Topic {}:\t".format(topic[0]), end="")
        for tup in topic[1]:
            print("{}, ".format(tup[0], tup[1]), end="")
        print()

Let's try to find 5 topics in the corpus using the Latent Dirichlet Allocation model. Note that running the algorithm can take a bit of time, due to the number of passes. To avoid having to train the model multiple times, the topics are stored in two files. It is also better to discuss to topics, because they are not exactly the same between multiple runs. Increasing the number of passes (default is 1) is good because it increases accuracy of topic modeling.

In [ ]:
try:
    data = pickle.load(open('5_topics.p', 'rb'))
except:
    model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=50)
    data = model.show_topics(num_topics=5, num_words=10, formatted=False)
    pickle.dump(data, open('5_topics.p', 'wb'))

format_topics(data)

As we can see, the topics are generic. They are about Obama, diplomacy and politics in general. For example, topic 0 is about elections ('elect', 'vote') and american politics ('democrat', 'republican', 'parti', 'senat'). We can compare these topics with the one generated by the LDA model when looking for 50 topics.

In [ ]:
try:
    data = pickle.load(open('50_topics.p', 'rb'))
except:
    model = LdaModel(corpus, num_topics=50, id2word=dictionary, passes=50)
    data = model.show_topics(num_topics=50, num_words=10, formatted=False)
    pickle.dump(data, open('50_topics.p', 'wb'))
    
format_topics(data)

As expected, they are more diverse. Instead of having one topic for diplomacy, we have one topic about pakistan (27), one about china (24), one about israel and palestinia (7), etc. There are also topics about recent events: North Korea testing nuclear weapons (27), attack in Benghazi (26), etc. We could say that 50 topics is too much because some topics seem to be not related to anything (2, 49, ...), but it is not necessarily true since some topics (Russia for example) do not appear at each run of the LDA model. Note that his can also be because the word 'Russia' is not important enough to be in the top ten words of any topic.

In [ ]:
try:
    data = pickle.load(open('25_topics.p', 'rb'))
except:
    model = LdaModel(corpus, num_topics=25, id2word=dictionary, passes=50)
    data = model.show_topics(num_topics=25, num_words=10, formatted=False)
    pickle.dump(data, open('25_topics.p', 'wb'))
    
format_topics(data)

When limited to 25, the topics are more centered around US politics ('democrat', 'republican', 'vote', 'elect', 'senat', 'bill', 'parti', 'presid', ...), meetings ('call', 'talk', 'tomorrow', 'work', 'offic', 'meet', 'room', ...) and middle east ('iran', 'israel', 'border', 'jewish', 'palestinian', 'ahmadinejad', 'peac', ...)

The best number of topics to look for actually depends on how deep a person wants to go. If set to 5, then only generic topics will emerge with not much information, and it probably won't be enough for most cases. With 50, more detailed topics appear, specialized for different countries the US are working with. But with these also come less meaningful topics. A good compromise would be to find a value between 5 and 25. For example, with 25, the topics are not too generic, and there aren't too much missing data.

# Bonus - Communication graph